In [1]:
import sys
sys.path.append('../code')
from resnet import *
from funcs import *
from cifar_very_tiny import *
from cifar_dataset import *    
import torch as t 
import numpy as np
from numpy import polyfit
from numpy import polyval
import tqdm
import matplotlib.pylab as plt
import matplotlib.cm as cm
import json
import hyperparams
from importlib import reload
from scipy.interpolate import interp1d
from PIL import Image
%matplotlib inline
plt.rcParams['figure.figsize']=(12,9)
plt.rcParams['font.size']= 20

In [2]:
# добавил в загрузку валидационную выборку
# обрати внимание, maxsize --- это размер совокпного обучения и валидации
# поэтому размер обучающей выборки совпадает с тем, что было до этого

train_loader_no_augumentation, test_loader = cifar10_loader(batch_size=128, split_train_val=False,
                                                                             maxsize=128*100, use_aug=False)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
epoch_num = 50
# epoch_num = 50

run_num = 1 # количество запусков эксперимента

# версия нужна, чтобы различать старые и новые результаты экспериментов. 
# менять нужно каждый раз, когда есть хотя бы незначительные изменения в эксперименте
experiment_version = '1'

validate_every_epoch = 5 

# train_splines_every_epoch = 5 # каждые 5 эпох отслеживать траекторию гиперпараметров
# train_splines_every_epoch = 2
# train_splines_every_epoch = 3
train_splines_every_epoch = 10

# размер мини-эпохи в батчах, за которую у нас производится либо обучение спайлов, либо их использование
mini_epoch_size = 10

start_beta = 0.5
start_temp  = 1.0

In [5]:
student = Cifar_Very_Tiny(10).cuda()
teacher = ResNet18(10).cuda() 
#scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.5)   
mu_feat = nn.Linear(512, 64).cuda()
log_sigma_feat = torch.nn.Parameter(torch.zeros(1).cuda())
optim = torch.optim.Adam(list(student.parameters()) + list(mu_feat.parameters()) + [log_sigma_feat])    

crit = nn.CrossEntropyLoss()

for e in range(25):
    tq = tqdm.tqdm(train_loader_no_augumentation)
    losses = []
            
    for batch_id, (x,y) in enumerate(tq):
        x = x.to(device)
        y = y.to(device)
        optim.zero_grad()
        student_feat, student_logits = student.get_features(x, [3,4])
        teacher_feat, teacher_logits = teacher.get_features(x, [3,4])
        class_loss = crit(student_logits, y)
        sigma2 = torch.log(1+torch.exp(log_sigma_feat))
        feat_loss = ((mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/sigma2 + torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
        loss = class_loss * 0.9# + feat_loss * 0.1
        
        loss.backward()
        optim.step()
        losses.append(loss.detach().cpu().numpy())
        tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))      
    student.eval()
    ac = float(accuracy(student, test_loader))
    print(e, ac)
    student.train()
    with open('acc_naive.txt','a') as out:
        out.write('{}: {}\n'.format(e, ac))

current loss:1.4524468183517456: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.54it/s]


0 0.4010999798774719


current loss:1.3158849477767944: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.59it/s]


1 0.4587000012397766


current loss:1.232130765914917: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.56it/s]


...

22 0.5221999883651733


current loss:0.744619607925415: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.49it/s]


23 0.5209000110626221


current loss:0.7325599789619446: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.48it/s]


24 0.51910001039505


In [5]:
crit = nn.CrossEntropyLoss()
for lam1 in [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.5]:
    student = Cifar_Very_Tiny(10).cuda()
    teacher = ResNet18(10).cuda() 
    teacher.load_state_dict(torch.load('../2021-Project-84/code/resnet18_cifar10.model?raw=true'))
    #scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.5)   
    mu_feat = nn.Linear(512, 64).cuda()
    log_sigma_feat = torch.nn.Parameter(torch.zeros(1).cuda())

    #mu_logit = nn.Linear(10, 10).cuda()
    #log_sigma_logit = torch.nn.Parameter(torch.zeros(1).cuda())


    optim = torch.optim.Adam(list(student.parameters()) + list(mu_feat.parameters()) + [log_sigma_feat])    


    for e in range(25):
        tq = tqdm.tqdm(train_loader_no_augumentation)
        losses = []

        for batch_id, (x,y) in enumerate(tq):
            x = x.to(device)
            y = y.to(device)
            optim.zero_grad()
            student_feat, student_logits = student.get_features(x, [3,4])
            teacher_feat, teacher_logits = teacher.get_features(x, [3,4])
            class_loss = crit(student_logits, y)
            sigma2 = torch.log(1+torch.exp(log_sigma_feat))
            feat_loss = ((mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/(2*sigma2) + 0.5*torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
            #logit_loss =((mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/(2*sigma2) + 0.5*torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
            #lam1 = 0.5
            loss = class_loss * (1.0-lam1) + feat_loss * lam1


            loss.backward()
            optim.step()
            losses.append(loss.detach().cpu().numpy())
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))      
        #scheduler.step()
        student.eval()
        ac = float(accuracy(student, test_loader))
        print(e, ac)
        student.train()
        with open('acc_mi.txt','a') as out:
            out.write('{}: {}: {}\n'.format(lam1, e, ac))

current loss:1.62313711643219: 100%|██████████| 100/100 [00:39<00:00,  2.52it/s] 


0 0.3920999765396118


current loss:1.4104820489883423: 100%|██████████| 100/100 [00:39<00:00,  2.52it/s]


1 0.47110000252723694


current loss:1.302858591079712: 100%|██████████| 100/100 [00:40<00:00,  2.50it/s]


...

22 0.20069999992847443


current loss:-327.21221923828125: 100%|██████████| 100/100 [00:38<00:00,  2.57it/s]


23 0.20059999823570251


current loss:-340.2093200683594: 100%|██████████| 100/100 [00:38<00:00,  2.58it/s]


24 0.20430000126361847


tensor([0.6931], device='cuda:0', grad_fn=<LogBackward0>)